## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="indy7gripper"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210920-194228', '20210920-224003', '20210921-014626', '20210921-044409', '20210921-074756', '20210921-103631', '20210921-134210', '20210921-165251', '20210921-200258', '20210921-230351']
['20210922-015645', '20210922-050510', '20210922-080941', '20210922-112600', '20210922-142917']
['20211003-203350', '20211003-205429', '20211003-211231', '20211003-212833', '20211003-214307', '20211003-220140', '20211003-221829', '20211003-223707', '20211003-225430', '20211003-231531']
['20211003-232937', '20211003-234739', '20211004-000211', '20211004-001937', '20211004-003939']


In [5]:
data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

data_pairs_train_failmore = []
for dataset in DATASET_TRAIN_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_failmore)))        
        
        
data_pairs_test_failmore = []
for dataset in DATASET_TEST_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_failmore)))        

train set: 10002
test set: 5005
train fail set: 10032
test fail set: 5016


In [6]:
N_max_train = 10000
N_max_test = 5000

print("=== mix data ===")
data_pairs_train_mixed = mix_data_pairs(
    data_pairs_train_filtered, data_pairs_train_failmore, N_max_train)
data_pairs_test_mixed = mix_data_pairs(
    data_pairs_test_filtered, data_pairs_test_failmore, N_max_test)

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]
data_pairs_train_failmore = data_pairs_train_failmore[:N_max_train]
data_pairs_test_failmore = data_pairs_test_failmore[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))
print("failmore data: {} / {}".format(
    len(data_pairs_train_failmore), 
    len(data_pairs_test_failmore)))
print("mixed data: {} / {}".format(
    len(data_pairs_train_mixed), 
    len(data_pairs_test_mixed)))

=== mix data ===
filtered success: 5000
success from failmore: 0
dataset fails: 5000
dataset all: 10000
filtered success: 2500
success from failmore: 0
dataset fails: 2500
dataset all: 5000
filtered data: 10000 / 5000
failmore data: 10000 / 5000
mixed data: 10000 / 5000


## Train

In [7]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

    
def train_script(data_pairs_train, data_pairs_test, robot_model_root,
                 EPOCHS_S=0, EPOCHS_E=15, BATCH_SIZE=16, LOG_STEP=100):
    current_time = get_now()
    logpath = os.path.join(robot_model_root, current_time)
    try_mkdir(logpath)
    train_log_dir = os.path.join(logpath, 'train')
    test_log_dir = os.path.join(logpath, 'test')
    model_log_dir = os.path.join(logpath, 'model_{}/')
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    shutil.copy(os.path.join(
        PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
    print(f'Log path: {logpath}')
    
    N_train = len(data_pairs_train)
    N_test = len(data_pairs_test)
    random.shuffle(data_pairs_test)
    res_list = []
    
    for epoch in range(EPOCHS_S, EPOCHS_E):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        random.shuffle(data_pairs_train)
        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_train:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_test:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        model.save(model_log_dir.format(epoch + 1))

        print("")
        print("=================================================================")
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )
        print("=================================================================")
        print("")
        res_list.append((float(test_accuracy.result())*100, float(test_loss.result())))
    return res_list

## train filtered data

In [9]:
ROBOT_MODEL_ROOT_DIV = "{}-filtered".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_filtered, data_pairs_test_filtered

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.4493720233440399, Accuracy: 80.72999572753906, Test Loss: 0.36350518465042114, Test Accuracy: 84.49519348144531



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.16330112516880035, Accuracy: 93.81999969482422, Test Loss: 0.401736855506897, Test Accuracy: 84.9959945678711



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered/20211004-135213/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.16103583574295044, Accuracy: 94.05500030517578, Test Loss: 0.39109113812446594, Test Accuracy: 84.364990234375

/home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-filtered acc/loss list:
1: 84.5 %, 0.364 
2: 85.18 %, 0.384 
3: 85.12 %, 0.346 
4: 84.16 %, 0.392 
5: 86.79 %, 0.326 
6: 85.95 %, 0.358 
7: 87.81 %, 0.305 
8: 87.75 %, 0.317 
9: 87.1 %, 0.311 
10: 87.87 %, 0.336 
11: 87.22 %, 0.312 
12: 86.96 %, 0.331 
13: 82.28 %, 0.396 
14: 86.35 %, 0.376 
15: 85.94 %, 0.333 
16: 86.88 %, 0.365 
17: 84.63 %, 0.39 
18: 83.88 %, 0.388 
19: 85.0 %, 0.402 
20: 84.36 %, 0.391 


## failmore data

In [10]:
ROBOT_MODEL_ROOT_DIV = "{}-failmore".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_failmore, data_pairs_test_failmore

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.3660651445388794, Accuracy: 86.55500030517578, Test Loss: 0.13933596014976501, Test Accuracy: 93.5196304321289



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.02170170657336712, Accuracy: 99.22999572753906, Test Loss: 0.21229702234268188, Test Accuracy: 95.65304565429688



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore/20211004-141305/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.027196306735277176, Accuracy: 99.02999877929688, Test Loss: 0.20883026719093323, Test Accuracy: 95.59294891357422

/home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-failmore acc/loss list:
1: 93.52 %, 0.139 
2: 94.33 %, 0.112 
3: 96.53 %, 0.094 
4: 96.0 %, 0.107 
5: 95.29 %, 0.109 
6: 96.27 %, 0.092 
7: 96.49 %, 0.123 
8: 95.79 %, 0.099 
9: 95.78 %, 0.126 
10: 96.25 %, 0.107 
11: 96.22 %, 0.111 
12: 95.89 %, 0.169 
13: 95.78 %, 0.157 
14: 95.69 %, 0.199 
15: 95.75 %, 0.156 
16: 95.98 %, 0.134 
17: 96.4 %, 0.166 
18: 95.98 %, 0.185 
19: 95.65 %, 0.212 
20: 95.59 %, 0.209 


## balanced data

In [11]:
ROBOT_MODEL_ROOT_DIV = "{}-balanced".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_mixed, data_pairs_test_mixed

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV,
                        EPOCHS_S=0, EPOCHS_E=20)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

Log path: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_1/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_1/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 1, Loss: 0.3283974826335907, Accuracy: 89.73500061035156, Test Loss: 0.256118506193161, Test Accuracy: 92.87860107421875



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_2/assets


...

INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_19/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 19, Loss: 0.06849873065948486, Accuracy: 97.97999572753906, Test Loss: 0.1677611768245697, Test Accuracy: 94.7215576171875



INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_20/assets


INFO:tensorflow:Assets written to: /home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced/20211004-143359/model_20/assets
/home/rnb/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



Epoch 20, Loss: 0.06499546766281128, Accuracy: 97.90499877929688, Test Loss: 1.2697595357894897, Test Accuracy: 90.224365234375

/home/rnb/Projects/rnb-planning/model/latticized/indy7gripper-balanced acc/loss list:
1: 92.88 %, 0.256 
2: 93.87 %, 0.474 
3: 94.74 %, 2.073 
4: 92.66 %, 1.555 
5: 91.5 %, 10.513 
6: 95.23 %, 0.142 
7: 94.64 %, 0.182 
8: 95.36 %, 0.14 
9: 95.35 %, 0.144 
10: 94.96 %, 0.157 
11: 95.72 %, 0.137 
12: 95.94 %, 0.137 
13: 95.7 %, 0.134 
14: 95.18 %, 0.155 
15: 94.98 %, 0.213 
16: 91.57 %, 1.968 
17: 93.61 %, 0.217 
18: 92.68 %, 0.341 
19: 94.72 %, 0.168 
20: 90.22 %, 1.27 


# Train results - Indy7, failmore data from before

#### filtered: 20211004-135213
```
1: 84.5 %, 0.364 
2: 85.18 %, 0.384 
3: 85.12 %, 0.346 
4: 84.16 %, 0.392 
5: 86.79 %, 0.326 
6: 85.95 %, 0.358 
7: 87.81 %, 0.305 
8: 87.75 %, 0.317 
9: 87.1 %, 0.311 
10: 87.87 %, 0.336 
11: 87.22 %, 0.312 
12: 86.96 %, 0.331 
13: 82.28 %, 0.396 
14: 86.35 %, 0.376 
15: 85.94 %, 0.333 
16: 86.88 %, 0.365 
17: 84.63 %, 0.39 
18: 83.88 %, 0.388 
19: 85.0 %, 0.402 
20: 84.36 %, 0.391 
```

#### failmore: 20211004-141305
```
1: 93.52 %, 0.139 
2: 94.33 %, 0.112 
3: 96.53 %, 0.094 
4: 96.0 %, 0.107 
5: 95.29 %, 0.109 
6: 96.27 %, 0.092 
7: 96.49 %, 0.123 
8: 95.79 %, 0.099 
9: 95.78 %, 0.126 
10: 96.25 %, 0.107 
11: 96.22 %, 0.111 
12: 95.89 %, 0.169 
13: 95.78 %, 0.157 
14: 95.69 %, 0.199 
15: 95.75 %, 0.156 
16: 95.98 %, 0.134 
17: 96.4 %, 0.166 
18: 95.98 %, 0.185 
19: 95.65 %, 0.212 
20: 95.59 %, 0.209 
```

#### balanced: 20211004-143359
```
1: 92.88 %, 0.256 
2: 93.87 %, 0.474 
3: 94.74 %, 2.073 
4: 92.66 %, 1.555 
5: 91.5 %, 10.513 
6: 95.23 %, 0.142 
7: 94.64 %, 0.182 
8: 95.36 %, 0.14 
9: 95.35 %, 0.144 
10: 94.96 %, 0.157 
11: 95.72 %, 0.137 
12: 95.94 %, 0.137 
13: 95.7 %, 0.134 
14: 95.18 %, 0.155 
15: 94.98 %, 0.213 
16: 91.57 %, 1.968 
17: 93.61 %, 0.217 
18: 92.68 %, 0.341 
19: 94.72 %, 0.168 
20: 90.22 %, 1.27 
```

## New train 2021-06
* **[IMPORTANT] 이전에 Tej 사용 틀린 점 보완 했음** 

### Panda
* 20210606-080139 : first balanced train trial, 96.6% 달성 / 이전 데이터와 collision boundary shape 차이일 수 있음
* 20210606-082302 : 이전 데이터와 전부 섞어서 collision boundary shape 차이 영향 확인 - 87% 가량, 영향 확실, 재수집 필요
* 20210606-191455 : 새로 모았어도 분리 학습 하니 시작부터 99%, 문제 있음
* 20210606-194148 : 새로 모은거 분리 학습, 테스트만 순수 기존 데이터로 하니 59%, 문제 확실
* 20210606-201525 : 새 데이터만 학습, 96%? 하지만 fail bias 돼 있음.
* 20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

### Indy7
* 20210607-224401 : 새 데이터만 학습, test set은 bias 제거함 ep12 79.5% 수준
* 20210610-052216 : 조인트 리밋 해제한 데이터만 학습, test set은 bias 제거, ep12 92% 수준

## Load & test

20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

In [12]:
def get_tf_functions(model_root, model_folder, epoch_folder):
    # last_model = '20210606-204309' # panda
    # last_model = '20210610-052216' # indy
    # last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
    logpath = os.path.join(model_root, model_folder)

    # epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
    # print(epochlist)
    # last_save = epochlist[-1]
    model_log_dir = os.path.join(logpath, epoch_folder)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    return model, inference, calc_loss

def test_script(data_pairs_test, model_root, model_folder, epoch_folder):
    BATCH_SIZE = 1
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()
    
    model, inference, calc_loss= get_tf_functions(model_root, model_folder, epoch_folder)

    grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
    res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])


    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]

    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)

    TP_FN_FP_TN = (np.sum(np.logical_and(res_list, label_list)), np.sum(np.logical_and(np.logical_not(res_list), label_list)), 
                   np.sum(np.logical_and(res_list, np.logical_not(label_list))), np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list))))
        
    return label_list, res_list

# Test all

In [13]:
print("========= Filtered =========")
model_foler, epoch_folder = '20211004-135213', 'model_10'
filtered_res = []
filtered_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))

print("========= Failmore =========")
model_foler, epoch_folder = '20211004-141305', 'model_3'
failmore_res = []
failmore_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))

print("========= Balanced =========")
model_foler, epoch_folder = '20211004-143359', 'model_12'
balanced_res = []
balanced_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))

print("========= Filtered =========")
print("filtered:")
print_confusion_mat(*filtered_res[0])
print("failmore:")
print_confusion_mat(*filtered_res[1])
print("balanced:")
print_confusion_mat(*filtered_res[2])
print("========================\n")

print("========= Failmore =========")
print("filtered:")
print_confusion_mat(*failmore_res[0])
print("failmore:")
print_confusion_mat(*failmore_res[1])
print("balanced:")
print_confusion_mat(*failmore_res[2])
print("========================\n")

print("========= Balanced =========")
print("filtered:")
print_confusion_mat(*balanced_res[0])
print("failmore:")
print_confusion_mat(*balanced_res[1])
print("balanced:")
print_confusion_mat(*balanced_res[2])
print("========================\n")

========= Filtered =========


test step - 5000/5000        
Test Loss: 0.33615586161613464 
Test Accuracy: 87.64 
TP / FN / ACC: 3588, 166, 95.58
FP / TN / ACC: 452, 794, 63.72
PACC / NACC / TACC: 88.81, 82.71, 87.64


inference: 	54049.0 ms/5000 = 10.81 ms (10.338/16.512)



test step - 5000/5000        
Test Loss: 1.1857982873916626 
Test Accuracy: 44.0 
TP / FN / ACC: 325, 7, 97.89
FP / TN / ACC: 2793, 1875, 40.17
PACC / NACC / TACC: 10.42, 99.63, 44.0


inference: 	52682.0 ms/5000 = 10.536 ms (10.108/13.419)



test step - 5000/5000        
Test Loss: 0.7540988922119141 
Test Accuracy: 67.28 
TP / FN / ACC: 2378, 122, 95.12
FP / TN / ACC: 1514, 986, 39.44
PACC / NACC / TACC: 61.1, 88.99, 67.28


inference: 	52418.0 ms/5000 = 10.484 ms (10.132/16.722)

========= Failmore =========


test step - 5000/5000        
Test Loss: 0.861128032207489 
Test Accuracy: 67.92 
TP / FN / ACC: 2343, 1411, 62.41
FP / TN / ACC: 193, 1053, 84.51
PACC / NACC / TACC: 92.39, 42.74, 67.92


inference: 	52869.0 ms/5000 = 10.574 ms (10.13/14.065)



test step - 5000/5000        
Test Loss: 0.09423182159662247 
Test Accuracy: 96.58 
TP / FN / ACC: 228, 104, 68.67
FP / TN / ACC: 67, 4601, 98.56
PACC / NACC / TACC: 77.29, 97.79, 96.58


inference: 	53187.0 ms/5000 = 10.637 ms (10.155/13.913)



test step - 5000/5000        
Test Loss: 0.5368154048919678 
Test Accuracy: 80.47999999999999 
TP / FN / ACC: 1557, 943, 62.28
FP / TN / ACC: 33, 2467, 98.68
PACC / NACC / TACC: 97.92, 72.35, 80.48


inference: 	52586.0 ms/5000 = 10.517 ms (10.058/14.167)

========= Balanced =========


test step - 5000/5000        
Test Loss: 0.6749281883239746 
Test Accuracy: 79.52 
TP / FN / ACC: 3687, 67, 98.22
FP / TN / ACC: 957, 289, 23.19
PACC / NACC / TACC: 79.39, 81.18, 79.52


inference: 	52734.0 ms/5000 = 10.547 ms (10.102/20.12)



test step - 5000/5000        
Test Loss: 0.20364302396774292 
Test Accuracy: 93.10000000000001 
TP / FN / ACC: 327, 5, 98.49
FP / TN / ACC: 340, 4328, 92.72
PACC / NACC / TACC: 49.03, 99.88, 93.1


inference: 	54066.0 ms/5000 = 10.813 ms (10.208/14.704)



test step - 5000/5000        
Test Loss: 0.13666516542434692 
Test Accuracy: 95.89999999999999 
TP / FN / ACC: 2453, 47, 98.12
FP / TN / ACC: 158, 2342, 93.68
PACC / NACC / TACC: 93.95, 98.03, 95.9


inference: 	52524.0 ms/5000 = 10.505 ms (10.134/14.324)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 3588 	 	 166 	 	 95.58%
GN 	 452 	 	 794 	 63.72%
AL 	 88.81% 	 82.71% 	 87.64%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 325 	 	 7 	 	 97.89%
GN 	 2793 	 	 1875 	 40.17%
AL 	 10.42% 	 99.63% 	 44.00%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2378 	 	 122 	 	 95.12%
GN 	 1514 	 	 986 	 39.44%
AL 	 61.10% 	 88.99% 	 67.28%

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2343 	 	 1411 	 	 62.41%
GN 	 193 	 	 1053 	 84.51%
AL 	 92.39% 	 42.74% 	 67.92%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 228 	 	 104 	 	 68.67%
GN 	 67 	 	 4601 	 98.56%
AL 	 77.29% 	 97.79% 	 96.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1557 	 	 943 	 	 62.28%
GN 	 33 	 	 2467 	 98.68%
AL 	 97.92% 	 72.

### Test result - indy
```
inference: 	120561.0 ms/5000 = 24.112 ms (9.798/31.185)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 3588 	 	 166 	 	 95.58%
GN 	 452 	 	 794 	 63.72%
AL 	 88.81% 	 82.71% 	 87.64%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 325 	 	 7 	 	 97.89%
GN 	 2793 	 	 1875 	 40.17%
AL 	 10.42% 	 99.63% 	 44.00%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2378 	 	 122 	 	 95.12%
GN 	 1514 	 	 986 	 39.44%
AL 	 61.10% 	 88.99% 	 67.28%
========================

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2343 	 	 1411 	 	 62.41%
GN 	 193 	 	 1053 	 84.51%
AL 	 92.39% 	 42.74% 	 67.92%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 228 	 	 104 	 	 68.67%
GN 	 67 	 	 4601 	 98.56%
AL 	 77.29% 	 97.79% 	 96.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1557 	 	 943 	 	 62.28%
GN 	 33 	 	 2467 	 98.68%
AL 	 97.92% 	 72.35% 	 80.48%
========================

========= Balanced =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 3687 	 	 67 	 	 98.22%
GN 	 957 	 	 289 	 23.19%
AL 	 79.39% 	 81.18% 	 79.52%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 327 	 	 5 	 	 98.49%
GN 	 340 	 	 4328 	 92.72%
AL 	 49.03% 	 99.88% 	 93.10%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2453 	 	 47 	 	 98.12%
GN 	 158 	 	 2342 	 93.68%
AL 	 93.95% 	 98.03% 	 95.90%
========================
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()